In [1]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

#//*** create model output path if needed
output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

output_path = "tts_train_dir"
training_dir = "training"
output_path = os.path.join(os.path.join(os.getcwd(), output_path))
output_path
training_dir = os.path.join(os.path.join(os.getcwd(), training_dir)).replace("\\","/")
print(str(training_dir))
t_path = str(output_path).replace("\\","/")
t_path
print(t_path)

import os
import wave
fulldir = os.path.join(output_path,"wavs")
#//**** Get the sample rate of the first WAV file, assume all files share the same sample_rate
for file_name in os.listdir(fulldir):
    with wave.open(os.path.join(fulldir,file_name), "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        print(file_name,frame_rate)
        break
    
    

C:/Users/stonk013/KGO_Projects/tts/ama_model/training
C:/Users/stonk013/KGO_Projects/tts/ama_model/tts_train_dir
ama-01.wav 48000


In [2]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.tacotron2_config import Tacotron2Config
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

# from TTS.tts.datasets.tokenizer import Tokenizer



dataset_config = BaseDatasetConfig(
    #formatter="vctk", meta_file_train="", language="en-us", path=os.path.join(os.path.join(output_path,"wavs"))
    formatter="ljspeech", meta_file_train="metadata.csv", language="en-us", path=output_path
)

audio_config = BaseAudioConfig(
    sample_rate=frame_rate,
    do_trim_silence=True,
    trim_db=60.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=8000,
    spec_gain=1.0,
    log_func="np.log",
    ref_level_db=20,
    preemphasis=0.0,
)

config = Tacotron2Config(  # This is the config that is saved for the future use
    audio=audio_config,
    batch_size=64,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    ga_alpha=0.0,
    decoder_loss_alpha=0.25,
    postnet_loss_alpha=0.25,
    postnet_diff_spec_alpha=0,
    decoder_diff_spec_alpha=0,
    decoder_ssim_alpha=0,
    postnet_ssim_alpha=0,
    r=2,
    attention_type="dynamic_convolution",
    double_decoder_consistency=False,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=True,
    mixed_precision=False,
    output_path=output_path,
    datasets=[dataset_config],
)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# If characters are not defined in the config, default characters are passed to the config
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=False,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# INITIALIZE THE MODEL
# Models take a config object and a speaker manager as input
# Config defines the details of the model like the number of layers, the size of the embedding, etc.
# Speaker manager is used by multi-speaker models.
model = Tacotron2(config, ap, tokenizer)

# INITIALIZE THE TRAINER
# Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training,
# distributed training, etc.
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

# AND... 3,2,1... 🚀
trainer.fit()

 > Training Environment:


 > Setting up Audio Processor...
 | > sample_rate:48000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 | > Found 12 files in C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir


 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Num. of CPUs: 8
 | > Num. of Torch Threads: 4
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir\run-August-10-2023_09+29PM-0000000

 > Model has 28259417 parameters

 > EPOCH: 0/1000
 --> C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir\run-August-10-2023_09+29PM-0000000


[*] Pre-computing phonemes...


  0%|          | 0/12 [00:00<?, ?it/s]

...

Text: fɪl sɛz ðə pɹɑbləm ɪz nɑt junik tə oʊklənd ænd sæn fɹænsɪskoʊ ivən smɔlɚ sɪtiz laɪk sæn ɹəmoʊn ɑɹ ɪkspɪɹiənsɪŋ haɪ veɪkənsi ɹeɪts fɚ kəmɚʃəl ɹil ɪsteɪt
cache_path C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir\phoneme_cache


100%|██████████| 12/12 [00:09<00:00,  1.21it/s]

 > TRAINING (2023-08-10 21:29:47) 


Text: æktɚ zɛndeɪə tədeɪ poʊstɪd ə mɛsɪd͡ʒ ɹɪmɛmbɚɪŋ hɚ koʊstɑɹ ænd fɛloʊ oʊklənd neɪtɪv æŋɡəs klaʊd klaʊdz fæmɪli ənaʊnst hɪz dɛθ jɛstɚdeɪ hɪz fæmɪli sɛd hi stɹʌɡəld wɪθ ðə dɛθ əv hɪz fɑðɚ wɪt͡ʃ hæpənd d͡ʒʌst læst wik ænd sʌfɚd fɹʌm mɛntəl hɛlθ ɪʃuz zɛndeɪə ænd klaʊd koʊ stɑɹd ɪn ðə eɪt͡ʃ bi oʊ dɹɑmə sɪɹiz jufɔɹiə ænd boʊθ ətɛndɪd oʊklənd skul fɚ ðə ɑɹts zɛndeɪ ɹoʊt ɔn ɪnstəɡɹæm kwoʊt aɪ noʊ pipəl juz ðɪs ɪkspɹɛʃən ɔftən wɛn tɔkɪŋ əbaʊt foʊks ðeɪ lʌv ðeɪ kʊd laɪt ʌp ɛni ɹum ðeɪ ɛntɚd bʌt bɔɪ lɛt mi tɛl ju hi wəz ðə bɛst æt ɪt aɪd laɪk tə ɹɪmɛmbɚ hɪm ðæt weɪ klaʊd wəz twɛnti faɪv jɪɹz oʊld wi hæv pʊt təɡɛðɚ ə hoʊl lɪst əv ɹisɔɹsɪz fɚ ɛniwʌn dilɪŋ wɪθ lɔs, ɡɹif ɔɹ mɛntəl hɛlθ ɪʃuz ju kən faɪnd loʊkəl ɹisɔɹsɪz baɪ ɡoʊɪŋ tə əbksɛvən njuz dɑt kɑm slæʃ teɪk ækʃən ænd ɪf ju nid ɪt ðə suɪsaɪd ænd kɹaɪsəs laɪflaɪn kən bi ɹit͡ʃt baɪ daɪlɪŋ naɪn eɪt eɪt


> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phonem

Traceback (most recent call last):
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\trainer\trainer.py", line 1806, in fit
    self._fit()
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\trainer\trainer.py", line 1758, in _fit
    self.train_epoch()
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\trainer\trainer.py", line 1485, in train_epoch
    for cur_step, batch in enumerate(self.train_loader):
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\torch\utils\data\dataloader.py", line 633, in __next__
    data = self._next_data()
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\torch\utils\data\dataloader.py", line 1345, in _next_data
    return self._process_data(data)
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\torch\utils\data\dataloader.py", line 1371, in _process_data
    data.reraise()
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\torch\_utils.py", line 644, in reraise
    r

AttributeError: 'tuple' object has no attribute 'tb_frame'